# 모듈 11: 루핑 워크플로우(Looping Workflow)

이 모듈에서는 **작업 -> 평가 -> 개선**의 과정을 반복하여 결과물의 품질을 점진적으로 높이는 **반복적(Iterative) 에이전트 시스템**을 배운다.

## 학습 목표

- **피드백 루프**: 생성(Generation)과 평가(Evaluation)를 순환하는 구조를 설계한다.
- **종료 조건(Termination)**: 특정 품질 기준(점수)을 충족하거나 최대 반복 횟수에 도달했을 때 루프를 멈춘다.
- **상태 활용**: 이전 단계의 결과물과 피드백을 다음 단계의 입력으로 전달하여 점진적 개선을 유도한다.

## 라이브러리 불러오기

먼저, 이 노트북이 빌드된 특정 버전의 Google Agent Development Kit (ADK)를 설치한다. 버전을 고정하면 코드가 항상 예상대로 작동한다.

In [1]:
import json
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

from google.adk.agents import Agent,SequentialAgent, LoopAgent
from google.adk.tools import google_search, ToolContext
from google.adk.tools.agent_tool import AgentTool
from google.adk.sessions import Session, InMemorySessionService
from google.genai.types import Content, Part
from google.adk.runners import Runner

## API 키 구성

다음으로 Google API 키를 안전하게 제공해야 한다. 이 코드는 키를 붙여넣을 수 있는 보안 입력 프롬프트를 생성한다. 그런 다음 키를 환경 변수로 설정하며, 이는 ADK가 요청을 인증하는 표준 방식이다.

In [2]:
load_dotenv()
MODEL = "gemini-2.5-flash"

## 워크플로 도구 정의

반복적인 워크플로를 위해 두 가지 핵심 함수가 필요하다. 회계사 에이전트가 계산을 수행하기 위한 `sum_costs`와 수정자(Refiner) 에이전트가 목표가 충족되었음을 알리고 루프를 종료하기 위한 `exit_loop` 함수다.

In [3]:
# 비용 합계 계산 도구
def sum_costs(costs: list[float]) -> float:
  """숫자 목록의 합계를 계산한다."""
  print(f"  [도구 호출] 다음 목록에 대한 sum_costs 실행: {costs}")
  return sum(costs)

# 루프 종료 신호 도구
def exit_loop(tool_context: ToolContext):
  """계획이 승인되고 예산 범위 내에 있을 때만 이 함수를 호출한다."""
  print(f"  [도구 호출] 예산 승인됨. 루프 종료: {json.dumps(tool_context.state.to_dict())}")
  
  # escalate 플래그를 True로 설정하여 루프를 탈출한다
  tool_context.actions.escalate = True
  return {"result" : "예산 승인됨. 계획 확정 중."}

## 도구 래퍼 생성

`cost_cutter_agent`가 내장된 Google 검색과 사용자 정의 `exit_loop` 함수를 모두 안정적으로 사용할 수 있도록, Google 검색 에이전트를 도구 에이전트(AgentTool)로 래핑한다.

In [5]:
# Google 검색 도구를 감싸는 간단한 에이전트 정의
google_search_agent = Agent(
    name="Google_Search_agent",
    model=MODEL,
    instruction="당신은 구글 검색 도구를 위한 래퍼일 뿐이다.",
    tools=[google_search]
)

# 에이전트를 도구로 변환하여 다른 에이전트가 사용할 수 있게 한다
google_search_tool = AgentTool(agent=google_search_agent)

## 제안자, 회계사, 비용 절감자 및 계획 검색자 에이전트 생성

### 제안자 (Proposer)
워크플로의 첫 번째 에이전트다. 사용자의 주제를 바탕으로 초기 계획을 제안하기 위해 한 번만 실행된다. 출력은 `current_plan` 변수에 저장된다.

### 회계사 (Accountant)
각 루프의 시작 부분에서 실행되는 에이전트다. `current_plan`을 가져와 `sum_costs` 도구를 사용하여 예산과 대조하고 비평(critique)을 출력한다.

### 비용 절감자 (Cost Cutter)
루프의 두 번째 단계 에이전트다. 비평을 입력으로 받는다. 예산이 충족되지 않으면 Google 검색 도구를 사용하여 더 저렴한 옵션을 찾는다. 예산이 충족되면 `exit_loop` 도구를 호출하여 프로세스를 종료한다.

### 계획 검색자 (Plan Retriever)
루프가 성공적으로 완료된 후 실행되는 유일한 임무를 가진 에이전트다. 세션 상태에서 최종 `current_plan`을 가져와 사용자에게 명확하게 제시한다.

In [7]:
# 완료 문구 정의 (이 문구가 나오면 루프가 종료된다)
COMPLETION_PHRASE = "계획이 예산 범위 내에 있다."

# 에이전트 1: 초기 고비용 계획을 제안한다 (1회 실행).
spending_proposer_agent = Agent(
    name="spending_proposer_agent",
    model=MODEL,
    tools=[google_search],
    instruction="""
    당신은 럭셔리 이벤트 기획자다. {{topic}}에 대해 고급 장소와 미식 케이터링 서비스를 찾아라.

    항목과 예상 비용이 포함된 JSON 객체를 출력하라. 예:
    {"venue": {"name": "The Ritz London", "cost": 10000}, "catering": {"name": "Gourmet Chefs Inc.", "cost": 5000}}
    """,
    output_key="current_plan",
)

# 에이전트 2 (루프 내): 계획을 비평하는 "회계사"다.
accountant_agent = Agent(
    name="accountant_agent",
    model="gemini-2.5-flash",
    tools=[sum_costs],
    instruction=f"""
    당신은 꼼꼼한 회계사다. 예산은 {{budget}}이다.
    현재 계획은 다음과 같다: {{current_plan}}

    계획에서 비용을 추출하고 `sum_costs` 도구를 사용하여 총액을 구하라.
    - 만약 총비용이 {{budget}}보다 크다면, 다음과 같이 비평하라: "이 계획은 [금액]만큼 예산을 초과했다. 더 저렴한 [항목]을 찾아라."
    - 그렇지 않다면, 정확히 다음 문구로 응답하라: '{COMPLETION_PHRASE}'
    """,
    output_key="critique"
)

# 에이전트 3 (루프 내): 계획을 수정하는 "비용 절감자"다.
cost_cutter_agent = Agent(
    name="cost_cutter_agent",
    model=MODEL,
    tools=[google_search_tool, exit_loop],
    instruction=f"""
    당신은 비용 절감 전문가다. 비평을 바탕으로 계획을 수정해야 한다.
    비평 내용은: {{critique}}
    현재 계획은: {{current_plan}}

    - 만약 비평이 '{COMPLETION_PHRASE}'라면, 반드시 인수 없이 `exit_loop` 도구를 호출해야 한다.
    - 그렇지 않다면, 비평을 읽고 너무 비싼 항목을 식별하라. 검색 도구를 사용하여 해당 항목에 대한 더 저렴한 대안을 찾아라.
      업데이트된 계획이 담긴 새로운 JSON 객체를 출력하라.
    """,
    output_key="current_plan"  # 이것은 다음 루프 반복을 위해 계획을 덮어쓴다.
)

# 에이전트 4: 루프가 끝난 후 최종 계획을 제시하기 위해 한 번 실행된다.
plan_retriever_agent = Agent(
    name="plan_retriever_agent",
    model=MODEL,
    instruction="""
    당신은 계획을 마무리하는 사람이다. 당신의 유일한 업무는 최종 승인된 계획을 제시하는 것이다.
    계획은 컨텍스트 변수 `{{current_plan}}`에서 사용할 수 있다.

    출력은 명확하고 읽기 쉬운 형식으로 제시된 최종 계획의 내용이어야 한다.
    """,
    tools=[]
)

## 루프 및 순차적 워크플로 조립

이제 에이전트 팀을 조립한다. `LoopAgent`는 비평 및 수정 주기를 처리하기 위해 생성된다. 그런 다음 최상위 `SequentialAgent`가 전체 "제안 -> 루프 -> 제시" 워크플로를 하나로 묶는다.

In [8]:
# 예산을 비평하고 수정할 루프 에이전트
budget_refinement_loop = LoopAgent(
    name="budget_refinement_loop",
    sub_agents=[accountant_agent, cost_cutter_agent],
    max_iterations=3, # 최대 3회 반복
)

# 전체 메인 워크플로 에이전트
budget_optimizer_workflow = SequentialAgent(
    name="budget_optimizer_workflow",
    sub_agents=[spending_proposer_agent, budget_refinement_loop, plan_retriever_agent]
)


## 실행 엔진 구축

이것은 쿼리를 실행하기 위한 헬퍼 함수이며, 이전 글과 변경되지 않았다. Runner를 초기화하고 이벤트를 스트리밍하는 핵심 ADK 로직을 처리한다.

In [9]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Runner를 초기화하고 주어진 에이전트 및 세션에 대해 쿼리를 실행한다."""
    print(f"\n에이전트 쿼리 실행 중: '{agent.name}', 세션: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user"),
            # 세션 상태의 예산과 주제를 컨텍스트로 전달한다
            state_delta={'budget': session.state.get('budget'), 'topic': session.state.get('topic')}
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"오류 발생: {e}"

    print("\n" + "-"*50)
    print("최종 응답:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response


## 세션 초기화 및 워크플로 실행

마지막으로 `InMemorySessionService`를 설정하고 메인 실행 블록을 정의한다. 세션 상태에서 초기 예산과 주제를 설정한 다음, 단일 호출로 전체 자율 워크플로를 시작한다.

In [10]:
# --- 세션 서비스 초기화 ---
# 이 하나의 서비스가 이 노트북의 모든 다른 세션을 관리한다.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

In [11]:
async def run_stateful_orchestrator():

  # 세션 생성
  session = await session_service.create_session(
        app_name=budget_optimizer_workflow.name,
        user_id=user_id
  )

  # 예산 및 주제 설정
  budget = 4000
  topic = "뉴욕에서 열리는 50명 규모의 AI 행사"
  
  # 세션 상태에 저장
  session.state["topic"] = topic
  session.state["budget"] = budget

  query = f"{topic}에 대한 계획을 찾아줘"
  print(f"사용자: {query}\n")
  
  # 워크플로 실행
  await run_agent_query(budget_optimizer_workflow, query, session, user_id)

# 전체 시스템 실행
await run_stateful_orchestrator()

사용자: 뉴욕에서 열리는 50명 규모의 AI 행사에 대한 계획을 찾아줘


에이전트 쿼리 실행 중: 'budget_optimizer_workflow', 세션: '1dc9553c-7160-4e67-bc70-1c489517eaea'...


  [도구 호출] 다음 목록에 대한 sum_costs 실행: [15000, 15000]
  [도구 호출] 다음 목록에 대한 sum_costs 실행: [900, 1000]
  [도구 호출] 예산 승인됨. 루프 종료: {"budget": 4000, "topic": "\ub274\uc695\uc5d0\uc11c \uc5f4\ub9ac\ub294 50\uba85 \uaddc\ubaa8\uc758 AI \ud589\uc0ac", "current_plan": "```json\n{\n  \"venue\": {\n    \"name\": \"Evalyn's Contemporary Tap House\",\n    \"description\": \"\ube0c\ub8e8\ud074\ub9b0 \uace0\uc640\ub108\uc2a4\uc5d0 \uc704\uce58\ud558\uba70 \ucd5c\ub300 100\uba85\uae4c\uc9c0 \uc218\uc6a9 \uac00\ub2a5\ud569\ub2c8\ub2e4. 50\uba85 \uaddc\ubaa8\uc758 \ud589\uc0ac\uc5d0\ub294 60\uba85\uae4c\uc9c0 \uc218\uc6a9 \uac00\ub2a5\ud55c \ud558\uc774\ud0d1 \uad6c\uc5ed\uc744 \uc774\uc6a9\ud560 \uc218 \uc788\uc73c\uba70, \uc608\uc57d \uc218\uc218\ub8cc\ub098 \ucd5c\uc18c \uc9c0\ucd9c\uc561\uc774 \uc5c6\ub294 \ub9e4\uc6b0 \uc800\ub834\ud55c \uc635\uc158\uc785\ub2c8\ub2e4. (\ud558\uc774\ud0d1 \uad6c\uc5ed\uc740 $900\uc758 \ucd5c\uc18c \uc9c0\ucd9c\uc561\uc774 \ud544\uc694\ud560 \uc218 \uc788\uc2b5\ub2c8\ub2e4.

The plan is within budget.

--------------------------------------------------

